# Neural Networks For Image and Metadata prediction

In [8]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torch.nn.functional as F
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
# ML and DL
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split as tts
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, balanced_accuracy_score, accuracy_score, precision_score, recall_score, roc_auc_score
import itertools
import pandas as pd
import os

In [31]:
class MyDataset(Dataset):
    def __init__(self, X, Y, folder = 'train'):
        # Load the images from the id on X dataframe and folder path
        #X is the dataframe with the ids and metadata
        #Y is the dataframe with the labels
        self.X = X
        self.Y = Y
        self.folder = folder

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        img_name = os.path.join(self.folder, self.X.iloc[index]['Id'] + '.jpg')
        image = plt.imread(img_name)
        image = torch.from_numpy(image).float()
        print("before")
        print(image.shape)
        image = image.permute(2,0,1)
        print("after")
        print(image.shape)
        # Resize the image to 224x224x3 (model input)
        image = F.interpolate(image.unsqueeze(0), size=224, mode="nearest").squeeze(0)
        
        label = torch.Tensor(self.Y.iloc[index])
        metadata = torch.Tensor(self.X.iloc[index, 1:])
        return {'image': image, 'target': label, 'metadata': metadata}

In [ ]:
def model_vis(cnn, title, train_loss, test_loss):
    
    min_index = np.argmin(test_loss)
    min_value = test_loss[min_index]
    
    # Plot the Training vs Epochs and Test vs Epochs
    plt.figure(figsize=(12,6))
    plt.subplot(1,1,1)
    plt.plot(range(len(train_loss)), train_loss, label='Training Loss')
    plt.plot(range(len(test_loss)), test_loss, label='Validation Loss')
    # Annotate the minimum value on the plot
    plt.annotate(f'Minimum: {min_index:.2f}', xy=(min_index, min_value), xytext=(min_index + 0.5, min_value + 0.1),
             arrowprops=dict(facecolor='black', shrink=0.05), fontsize=12)

    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.title(title + ' - Loss')

    plt.savefig("C:\\Users\\HP\\Desktop\\SLURM outing\\" + str(cnn) + "\\" + title + "_curve.png")
    plt.close()

    return None

In [ ]:
def test(dataloader, model, criterion, device):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)


  model.eval()
  test_loss = 0

  with torch.no_grad():
    for i, (images, labels) in enumerate(dataloader):
      images = images.to(device)

      #Prediction
      pred = model(images)
      #The decoder should result in the same image
      test_loss += criterion(pred, images).item()

    test_loss /= num_batches

  return test_loss

In [ ]:
def train(dataloader, model, criterion, optimizer, device):
  size = len(dataloader.dataset)
  model.train()
  tr_loss = 0.0
  tr_auc = 0.0
  tr_acc = 0.0
  tr_bal_acc = 0.0
  j = 0

  for i, (images, labels) in enumerate(dataloader):

      images = images.to(device)
      labels = labels.to(device)

      #zero the parameter gradient
      optimizer.zero_grad()

      #Forward pass
      # Checks the outputs given by the model at this training stage and compares it to the original labels with the desired loss criteria
      outputs = model(images)
      loss = criterion(outputs, images)

      #Bacward pass
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      tr_loss += loss.item()
      j+=1

  tr_loss /= j
  return tr_loss

In [ ]:
def metric(y_true, y_pred):

  y_true = y_true.numpy(force= True)
  y_pred = y_pred.numpy(force= True)

  # Get metrics for regression
  rmse = np.sqrt(mean_squared_error(y_true, y_pred))
  r2 = r2_score(y_true, y_pred)

  return rmse, r2

In [13]:
# Load the metadata file and concatenate it with the image data by id

train_data = pd.read_csv('train.csv')


X = train_data.drop('Pawpularity', axis=1) #except the last and the first(Id)
y = train_data['Pawpularity']/100.0

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = tts(X, y, test_size=0.2, random_state=42)

X.iloc[0, 1:]

Subject Focus    0
Eyes             1
Face             1
Near             1
Action           0
Accessory        0
Group            1
Collage          0
Human            0
Occlusion        0
Info             0
Blur             0
Name: 0, dtype: object

# CNN Regressor

In [15]:
class CNNRegressor(nn.Module):
    def __init__(self, input_size):
        super(CNNRegressor, self).__init__()
        kernel_1 = 3
        padding_1 = 0
        stride_1 = 1
        out_filters_1 = 16
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        out_1 = (input_size[0] - kernel_1 + padding_1 *2) / stride_1 + 1
        self.fc1 = nn.Linear(out_filters_1 * out_1 * out_1 , 1) 

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return x

In [32]:
# Hyperparameters
batch_size = 32
learning_rate = 0.001
epochs = 100

# Create instances of the custom dataset
train_dataset = MyDataset(X_train, y_train, 'train')
val_dataset = MyDataset(X_val, y_val, 'train')

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Get the input size of the model
images, targets, metadata = next(iter(train_loader))
input_size = images[0].shape
# Initialize the CNN regressor and define the loss function and optimizer
model = CNNRegressor(input_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for batch in train_loader:
        images, targets, metadata = batch['image'], batch['target'], batch['metadata']

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

    # Validation loop
    model.eval()
    with torch.no_grad():
        val_loss = 0.0
        for batch in val_loader:
            images, targets, metadata = batch['image'], batch['target'], batch['metadata']
            outputs = model(torch.Tensor(images))
            val_loss += criterion(outputs, targets).item()

        val_loss /= len(val_loader)
        print(f'Epoch {epoch + 1}/{epochs}, Validation Loss: {val_loss}')



before
torch.Size([810, 810, 3])
after
torch.Size([3, 810, 810])


TypeError: new(): data must be a sequence (got numpy.float64)

In [33]:
next(iter(train_loader))

before
torch.Size([956, 714, 3])
after
torch.Size([3, 956, 714])


TypeError: new(): data must be a sequence (got numpy.float64)